In [1]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance
import pandas as pd

/Users/mhdfarhanali/Documents/Instru RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Inisialisasi model embedding
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Hubungkan ke Qdrant
qdrant = QdrantClient(host="localhost", port=6333)
collection_name = "instru_collection"

print(f"Koneksi ke koleksi '{collection_name}' berhasil. Siap untuk query!")

Koneksi ke koleksi 'instru_collection' berhasil. Siap untuk query!


In [3]:
# Pertanyaan uji coba
query = "What is a transducer in instrumentation?"

# Buat embedding dari pertanyaan
query_vector = embedding_model.encode(query)

In [4]:
search_results = qdrant.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=3,
)

import textwrap

print("\nHasil pencarian paling relevan:\n")
for hit in search_results.points:
    content = hit.payload.get("page_content") or hit.payload.get("text", "")
    print(f"(score={hit.score:.3f})")
    print(textwrap.fill(content, width=100))
    print("\n" + "="*80 + "\n")


Hasil pencarian paling relevan:

(score=0.697)
A transducer converts one form of energy into another, often physical to electrical.


(score=0.697)
A transducer converts one form of energy into another, often physical to electrical.


(score=0.681)
Transducer A device that changes energy from one form to another. Transfer function An equation that
describes the relationship between the input and output of the function. Transmission The
transferring of information from one point to another. Transmitter A device that conditions the
signal received from a transducer so that it is suitable for sending to another location with
minimal loss of information. Turbine flow meter A flow-measuring device utilizing a turbine wheel.
Ultrasonic probe An




# Interpretasi

1. Tujuan
Notebook ini digunakan untuk menguji apakah data hasil ingestion sudah tersimpan dengan benar di database Qdrant, serta memastikan bahwa sistem bisa mencari informasi yang relevan berdasarkan pertanyaan (query) yang diberikan.
Tahap ini bisa dibilang sebagai “uji pencarian otak” dari Instru RAG, karena di sinilah sistem pertama kali mencoba menemukan konteks yang sesuai dari ribuan potongan teks (chunks) yang sudah dimasukkan sebelumnya.

2. Proses yang Dilakukan
Langkah-langkah yang dilakukan di tahap ini cukup sederhana tapi penting:

a. Koneksi ke Database
Pertama, sistem mencoba terhubung ke database Qdrant dan mengakses koleksi (collection) yang sudah dibuat sebelumnya, yaitu instru_collection.
Pesan yang muncul di terminal:
“Koneksi ke koleksi 'instru_collection' berhasil. Siap untuk query!”
Ini berarti database aktif dan sistem siap digunakan untuk melakukan pencarian informasi.

b. Membuat Query (Pertanyaan)
Selanjutnya, dilakukan pengujian dengan membuat sebuah pertanyaan contoh:
“What is a transducer in instrumentation?”
Pertanyaan ini dipilih karena sangat umum di bidang instrumentasi dan pasti ada di dalam isi buku Introduction to Instrumentation, Sensors, and Process Control.

c. Proses Pencarian (Retrieval)
Sistem kemudian mengubah pertanyaan tersebut menjadi vektor numerik menggunakan model embedding.
Lalu vektor pertanyaan dibandingkan dengan vektor-vektor yang ada di database untuk mencari potongan teks yang paling mirip secara semantik.
Dengan kata lain, sistem tidak mencari kata yang persis sama, tetapi mencari makna yang paling mirip.

d. Hasil Pencarian
Hasil yang muncul menunjukkan bahwa sistem berhasil menemukan beberapa potongan teks yang relevan dengan pertanyaan.
Berikut ringkasan hasilnya:

(score=0.697)
A transducer converts one form of energy into another, often physical to electrical.
dan

(score=0.681)
Transducer A device that changes energy from one form to another. Transfer function An equation that
describes the relationship between the input and output of the function...

Nilai score menunjukkan tingkat kemiripan antara pertanyaan dan teks yang ditemukan.
Semakin tinggi nilainya (mendekati 1.0), semakin relevan hasilnya.
Dari hasil di atas, terlihat bahwa sistem bisa menjawab dengan benar bahwa transducer adalah alat yang mengubah satu bentuk energi menjadi bentuk energi lain, biasanya dari energi fisik ke energi listrik.

3. Hasil Akhir
Dari hasil pengujian, bisa disimpulkan bahwa:
Sistem berhasil terhubung dengan Qdrant tanpa error.
Query atau pertanyaan bisa dijalankan dan menghasilkan potongan teks yang relevan.
Hasil pencarian sesuai konteks isi buku, artinya data di database sudah terindeks dengan benar.
Secara keseluruhan, tahap ini memastikan bahwa Instru RAG sudah “bisa berpikir” dengan membaca isi database-nya sendiri.

4. Kesimpulan
Tahap 2_index_data.ipynb merupakan langkah penting yang membuktikan bahwa proses ingestion sebelumnya berjalan dengan benar.
Sekarang sistem bisa melakukan pencarian konteks otomatis yang akan menjadi dasar bagi tahap selanjutnya, yaitu inference (menjawab pertanyaan secara penuh dengan bantuan LLM).
Dengan hasil ini, chatbot Instru RAG sudah memiliki kemampuan retrieval yang baik dan siap digabungkan dengan model AI untuk menghasilkan jawaban cerdas.

## Catatan Pribadi
Menurut saya, tahap ini seperti melatih chatbot untuk mencari jawaban di “otaknya sendiri”.
Awalnya saya ragu apakah sistem bisa menemukan bagian teks yang tepat, tapi setelah melihat hasilnya yang relevan dengan kata “transducer”, saya jadi lebih paham cara kerja RAG sebenarnya.
Saya juga belajar bahwa angka score di hasil pencarian itu menunjukkan seberapa mirip arti antara pertanyaan dan teks di database.
Melihat sistem berhasil menemukan penjelasan yang benar membuat saya cukup bangga karena ini berarti semua proses sebelumnya berhasil dilakukan dengan benar.